In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import tensorflow as tf
import re

pd.DataFrame.iteritems = pd.DataFrame.items

2023-09-29 08:21:41.470127: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
!pwd

/Deep Learning/capstone_project/notebooks/RPH-HAC-Prediction


In [2]:
def plot_vertical_categorical_data(column_name, df, top_n):
    df_counts = df[column_name].value_counts()
    top_X = df_counts.head(top_n)
    
    plt.figure(figsize=(10, 4))
    sns.barplot(x=top_X.index, y=top_X.values)
    plt.title('Top '+str(top_n)+' Most Common ' + column_name)
    plt.xlabel(column_name)
    plt.ylabel('Number of Admissions')
    plt.xticks(rotation=45, ha='right')  # Rotate the diagnosis names for better readability
    plt.tight_layout()
    plt.show()

def plot_horizontal_categorical_data(column_name, df, top_n):
    df_counts = df[column_name].value_counts()
    top_X = df_counts.head(top_n)
    
    plt.figure(figsize=(10, 8))
    sns.barplot(x=top_X.values, y=top_X.index)
    plt.title('Top '+str(top_n)+' Most Common ' + column_name)
    plt.ylabel(column_name)
    plt.xlabel('Number of Admissions')
    # You might not need rotation for y-axis labels, but if you do, adjust accordingly
    # plt.yticks(rotation=45, va='center')  
    plt.tight_layout()
    plt.show()
    
# function that performs basic regex operations on column of interest
def basic_regex(df, column_name):  
    df[column_name] = df[column_name].str.replace(',(?!\s)', ', ', regex=True)  # Ensure there's a space following a , for ,s that do not have a space 
    df[column_name] = df[column_name].str.replace('\(\s+', '(', regex=True) # remove one or more spaces that occur following an (
    df[column_name] = df[column_name].str.replace('\s+\)', ')', regex=True) # remove one or more spaces that occur before an )
    df[column_name] = df[column_name].str.replace('\s*/\s*', '/', regex=True)  # remove any spaces that occur directly before/after /
    df[column_name] = df[column_name].str.replace('\s+-\s+', '-', regex=True)  # remove spaces surrounding hyphens 
    df[column_name] = df[column_name].str.replace('-', ' ', regex=True)  # replace hyphens with blank spaces 
    df[column_name] = df[column_name].str.replace('[.;,-?]', '', regex=True).str.strip().str.lower()
    
# function to print the unique combinations of ITEMID and NAME
def itemid_name_unique_values(df, column_itemid, column_desc):
    df_sorted = df.sort_values(by=column_desc, ascending=True)  # sort the df by organism name
    unique_pairs = df_sorted.drop_duplicates(subset=[column_itemid, column_desc])  # drop duplicate rows based on the two columns
    
    # print the unique combinations of ITEMID and NAME
    for _, row in unique_pairs.iterrows():
        print(f"{column_itemid}: {row[column_itemid]}, {column_desc}: {row[column_desc]}")

In [3]:
micro = pd.read_csv('/Deep Learning/capstone_project/Data/MICROBIOLOGYEVENTS.csv')

# perform basic regex
basic_regex(micro, 'ORG_NAME')
basic_regex(micro, 'SPEC_TYPE_DESC')

# deal with na values
# micro['ORG_NAME'].fillna('no organism specified', inplace=True)
# micro['AB_NAME'].fillna('no AB', inplace=True)
# micro['INTERPRETATION'].fillna('NI', inplace=True)  # make na values equate to NI (No Interpretation)

# create a new column splitting the isolate_num into three groups, no_isolate 0, 1, and > 1 (2)
micro['ISOLATE_GROUP'] = np.where(micro['ISOLATE_NUM'].isna(), 0, np.where(micro['ISOLATE_NUM'] == 1, 1, 2))

In [5]:
micro_filtered = micro[['HADM_ID', 'ORG_NAME', 'SPEC_TYPE_DESC', 'INTERPRETATION']]
micro_filtered

,HADM_ID,ORG_NAME,SPEC_TYPE_DESC,INTERPRETATION
0,170324,pseudomonas aeruginosa,bronchoalveolar lavage,NaN
1,170324,NaN,sputum,NaN
2,170324,NaN,blood culture,NaN
3,170324,NaN,blood culture,NaN
4,170324,NaN,urine,NaN
...,...,...,...,...
631721,126090,aspergillus fumigatus,tissue,NaN
631722,126090,aspergillus fumigatus,tissue,NaN
631723,126090,aspergillus fumigatus,tissue,NaN
631724,126090,NaN,blood culture,NaN


In [45]:
prescriptions = pd.read_csv('/Deep Learning/capstone_project/Data/PRESCRIPTIONS.csv', low_memory=False) 

In [46]:
routes = {
    "Oral and Enteral": [
        "ORAL", "PO", "G TUBE", "J TUBE", "ENTERAL TUBE ONLY ? NOT ORAL",
        "OG", "NG", "NG/OG", "PO OR ENTERAL TUBE", "PO/NG", "PO/OG"],
    
    "Respiratory and Inhalation": [
        "AERO", "IH", "INHALATION", "NEB", "ET", "NU"],
    
    "Ocular and Otic": [
        "AS", "AU", "BOTH EARS", "LEFT EAR", "RIGHT EAR", 
        "OD", "OS", "OU", "BOTH EYES", "LEFT EYE", "RIGHT EYE"],
    
    "Dermal and Transdermal": [
        "ID", "SC", "SUBCUT", "TD", "BUCCAL", "BU", "SL", "AXILLARY"],
    
    "Intravenous and Intramuscular": [
        "IV", "IM", "IV BOLUS", "IV DRIP", "IVPCA", "IJ"],
    
    "Other Invasive Routes": [
        "ED", "IA", "IC", "IT", "IO", "IP", "IR", "IVT", "PR", "INTRAPERICARDIAL"],
    
    "Miscellaneous and Specific Methods": [
        "EX-VIVO", "LOCK", "IRR", "DIALYS", "DWELL"
    ]
}

In [47]:
def categorize_route(route):
    for key, values in routes.items():
        if route in values:
            return key
    return 'Unknown'  # to handle cases not in any of our lists

# Creating a new column 'ROUTE_GROUP' based on the mappings in the dictionary
prescriptions['ROUTE_GROUP'] = prescriptions['ROUTE'].apply(categorize_route)

In [48]:
prescriptions_filtered = prescriptions[['HADM_ID', 'DRUG_TYPE', 'ROUTE_GROUP']]
prescriptions_filtered.shape

(4156450, 3)

In [49]:
prescriptions_filtered = prescriptions[['HADM_ID', 'DRUG_TYPE', 'ROUTE_GROUP']]

# join micro and prescriptions

In [ ]:
micro_prescription = pd.merge(micro_filtered, prescriptions_filtered, on='HADM_ID', how='left')

In [50]:
micro_oh = pd.get_dummies(micro_filtered, columns=['ORG_NAME', 'SPEC_TYPE_DESC', 'ISOLATE_GROUP', 'INTERPRETATION'], prefix='', prefix_sep='')
micro_onehot = micro_oh.groupby('HADM_ID').sum().reset_index()  # convert true false to zero 0

In [51]:
micro_onehot

,HADM_ID,abiotrophiagranulicatella species,achromobacter (alcaligenes) dentrificans,achromobacter (alcaligenes) xylosoxidans ss dentrificans,acidfast bacilli,acinetobacter baumannii,acinetobacter baumannii complex,acinetobacter sp,acremonium species,actinomyces meyerii,...,PIPERACILLIN/TAZO,RIFAMPIN,TETRACYCLINE,TOBRAMYCIN,TRIMETHOPRIM/SULFA,VANCOMYCIN,I,P,R,S
0,100001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,100003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,100006,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,100007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,100009,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48735,199993,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48736,199994,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,0,0,5,4
48737,199995,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
48738,199998,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
# to do: pull the pre-processing, and add adjustments

prescriptions_oh = pd.get_dummies(prescriptions_filtered, columns=['DRUG_TYPE', 'ROUTE_GROUP'], prefix='', prefix_sep='')
prescriptions_onehot = prescriptions_oh.groupby('HADM_ID').sum().reset_index()
prescriptions_onehot.head()

,HADM_ID,ADDITIVE,BASE,MAIN,Dermal and Transdermal,Intravenous and Intramuscular,Miscellaneous and Specific Methods,Ocular and Otic,Oral and Enteral,Other Invasive Routes,Respiratory and Inhalation,Unknown
0,100001,0,33,59,12,65,0,0,15,0,0,0
1,100003,0,7,22,0,16,0,0,9,0,0,4
2,100006,0,5,70,3,19,0,0,34,0,19,0
3,100007,0,20,64,3,70,0,0,7,2,2,0
4,100009,0,17,105,21,53,0,0,45,2,1,0


In [66]:
micro_prescription = pd.merge(micro_onehot, prescriptions_onehot, on='HADM_ID', how='outer')

In [68]:
micro_prescription.isnull().sum()

HADM_ID                                                        0
abiotrophiagranulicatella species                           7559
achromobacter (alcaligenes) dentrificans                    7559
achromobacter (alcaligenes) xylosoxidans ss dentrificans    7559
acidfast bacilli                                            7559
                                                            ... 
Ocular and Otic                                             6083
Oral and Enteral                                            6083
Other Invasive Routes                                       6083
Respiratory and Inhalation                                  6083
Unknown                                                     6083
Length: 491, dtype: int64

In [70]:
micro_prescription.fillna(0, inplace=True)

In [72]:
micro_prescription.isnull().sum()

HADM_ID                                                     0
abiotrophiagranulicatella species                           0
achromobacter (alcaligenes) dentrificans                    0
achromobacter (alcaligenes) xylosoxidans ss dentrificans    0
acidfast bacilli                                            0
                                                           ..
Ocular and Otic                                             0
Oral and Enteral                                            0
Other Invasive Routes                                       0
Respiratory and Inhalation                                  0
Unknown                                                     0
Length: 491, dtype: int64

In [73]:
micro_prescription.to_csv('micro_prescription.csv', index=False)

Maybe do some imputation of variables